In [9]:
from openai import OpenAI
from typing import Literal, TypeAlias
import pandas as pd
import os
# get the current directory (folder)
python_directory = os.getcwd()
# extract the api key file and text
api_txt_file = open(rf"{python_directory}/api_key_private.txt", "r")
api_txt = api_txt_file.read()
# establish a client with the API key
client = OpenAI(
  api_key=api_txt
)

# Track the GPT conversation

In [11]:
ChatModel: TypeAlias = Literal[
    "gpt-4.1",
    "gpt-4.1-mini",
    "gpt-4.1-nano",
    "gpt-4.1-2025-04-14",
    "gpt-4.1-mini-2025-04-14",
    "gpt-4.1-nano-2025-04-14",
    "o4-mini",
    "o4-mini-2025-04-16",
    "o3",
    "o3-2025-04-16",
    "o3-mini",
    "o3-mini-2025-01-31",
    "o1",
    "o1-2024-12-17",
    "o1-preview",
    "o1-preview-2024-09-12",
    "o1-mini",
    "o1-mini-2024-09-12",
    "gpt-4o",
    "gpt-4o-2024-11-20",
    "gpt-4o-2024-08-06",
    "gpt-4o-2024-05-13",
    "gpt-4o-audio-preview",
    "gpt-4o-audio-preview-2024-10-01",
    "gpt-4o-audio-preview-2024-12-17",
    "gpt-4o-mini-audio-preview",
    "gpt-4o-mini-audio-preview-2024-12-17",
    "gpt-4o-search-preview",
    "gpt-4o-mini-search-preview",
    "gpt-4o-search-preview-2025-03-11",
    "gpt-4o-mini-search-preview-2025-03-11",
    "chatgpt-4o-latest",
    "gpt-4o-mini",
    "gpt-4o-mini-2024-07-18",
    "gpt-4-turbo",
    "gpt-4-turbo-2024-04-09",
    "gpt-4-0125-preview",
    "gpt-4-turbo-preview",
    "gpt-4-1106-preview",
    "gpt-4-vision-preview",
    "gpt-4",
    "gpt-4-0314",
    "gpt-4-0613",
    "gpt-4-32k",
    "gpt-4-32k-0314",
    "gpt-4-32k-0613",
    "gpt-3.5-turbo",
    "gpt-3.5-turbo-16k",
    "gpt-3.5-turbo-0301",
    "gpt-3.5-turbo-0613",
    "gpt-3.5-turbo-1106",
    "gpt-3.5-turbo-0125",
    "gpt-3.5-turbo-16k-0613",
]

def track_completions(
    messages:list, 
    model:ChatModel,
    history:list=list(),
    **kwargs):
    """
    Tracks the `completion` returns from the LLM for a list of messages. 
    The function feeds the LLM the history of all the prompts and replies.

    Parameters
    ----------
    `messages` : list of dictionary
        A list of message dictionaries. 
        Each dictionary must include a "role" key and a "content" key (see `client.chat.completions.create` documentation)
    
    Return
    ------
    dictionary
        "completion" key: a list of all the `ChatCompletion` objects from the LLM\n
        "reply" key: a list of the strings generated as the LLM replies\n
        "prompt" key: a list of the strings entered to the LLM\n
        "conversation" key: a list of the prompts and replies in order\n
    """
    messages_current = history
    completions = list()
    replies = list()
    conversations = list()
    conversations_dict = list()
    # iterate through the messages available
    for message in messages:
        ## append the prompt to the messages
        messages_current.append(message)
        ## append the prompt to the conversation with the role labelled
        conversations.append(f"{message["role"]}: {message["content"]}")
        conversations_dict.append(message)
        ## request a reply from the LLM
        completion = client.chat.completions.create(
            model=model,
            # model= kwargs["model"] if "model" in kwargs.keys() else None,
            store= kwargs["store"] if "store" in kwargs.keys() else None,
            messages=messages_current,
            max_tokens=kwargs["max_tokens"] if "max_tokens" in kwargs.keys() else None,
        )
        ## store the reply from the LLM in the messages format ("role" and "content")
        messages_reply = {
            "role": "assistant",
            "content": completion.choices[0].message.content
        }
        ## add the reply to the current messages the LLM should be aware of
        messages_current.append(messages_reply)
        ## append the outcomes to completions, replies and conversations
        completions.append(completion)
        replies.append(completion.choices[0].message.content)
        conversations.append(f"{"assistant"}: {completion.choices[0].message.content}")
        conversations_dict.append(messages_reply)
    # create the dictionary output and return it
    completion_reply = {
        "completion": completions,
        "reply": replies,
        "prompt": messages,
        "conversation": conversations,
        "conversation dictionary": messages_current
    }
    return completion_reply

# Provide the model description and get the python version of the model

In [18]:
messages = [
    {
        "role": "user",
        "content": f"""
        {open(rf"{python_directory}/Housing_ABM_description.txt", "r").read()}

        I want you to extract the behaviours in the sent description for each type of agent (with clear mathematical formulas as stated in the description).
        """
    },

    {
        "role": "user",
        "content": """
        I want you to create a model in python that includes all the behaviours and mathematical formulas you extracted.
        1. Each time step must represent 3 months. 
        2. I want the user interface to have a game-like aethetic (you can use pygame).
        3. I want you to the UI to have a graph on the right hand side showing the average sale price and the average rent price. The graph must be labelled with a legend, and must show the value of the average sale price and rent price in the latest time step.
        Assure the graph shows the mean prices of all the houses (not only transactions) and the mean rents, the graph must show each time step on the x axis and the value in british pound on the y axis. Make sure that the model updates these values every time step.
        4. Make sure you avoid the following error when drawing the graph raw_data = canvas.tostring_rgb()
               AttributeError: 'FigureCanvasAgg' object has no attribute 'tostring_rgb'. Did you mean: 'tostring_argb'
        5. Assure the houses that are newly built houses are always offered on the mortgage market for-sale. They do not need to have an owner to be on the mortgage market when they are first build.
        6. Assure that if a house is vacant, it is either offered for sale or for rent. This depends on the owner and their financial situation as state in the initial model description I sent.
        7. Make sure you use the following parameter values in the model
        
        Parameter & Value & source\\
        interest-rate (\(I_t\)) & 3.7\% & \citep{boe2023data}\\
        propensity-threshold & 0.2 & Yields 9\% landlords to total population as per Figure \ref{fig:sensitivity}-D3 \citep{benham2023} \\
        occupancy-ratio & 95\% & \\
        owners-to-tenants & 50\% & \\
        LTV (\(LTV_t\))& 90\% & \citep{boe2023data}\\
        mortgage-duration & 25 & \\
        rate-duration-M & 2 to 5 & \citep{Cloyne2019}\\
        rate-duration-BTL & 1 to 5 & \citep{boe2023BTL, mfb2020}\\
        mean-income & 30000 & \citep{ONS2023income}\\
        wage-increase & 0 & \citep{ONS2024wagerise}\\
        affordability (\(\alpha\)) & 33\% & \\
        savings-M & 20\% & \\
        savings-R & 5\% & \\
        homeless-period & 5 & \\
        search-length & 5 & \\
        construction-rate & 0.6\% & \citep{DLUHC2023}\\
        entry-rate & 4\% & \\
        exit-rate & 2\% & \\
        realtor-territory & 16 & \\
        price-drop-rate & 3\% & \\
        rent-drop-rate & 3\% & \\
        savings-threshold-M (\(\omega\)) & 2 & \\
        evict-threshold-M (\(\beta\)) & 1 & \\
        savings-threshold-R (\(\lambda\)) & 2 & \\
        evict-threshold-R (\(\gamma\)) & 1 & \\
        steps-per-year & 4 & \\
        
        ---

        For clarity, I want a complete model, any further extensions you propose must be out of the model description I previously sent.
        I want to be confident that the model you provide is not missing any behaviour or mathematcial formula in the description I previously sent.
        """
    }
]


completions = track_completions(
    messages=messages, 
    model="gpt-4.1-mini-2025-04-14",
    max_tokens=32768)

df = pd.DataFrame({"data": completions})
df.to_pickle(rf"History Prompts/GPT_4.1_mini_0.4.pkl")

for reply in completions["reply"]:
    print("###############################################################################################################\n")
    print(reply)


<>:13: SyntaxWarning: invalid escape sequence '\('
<>:13: SyntaxWarning: invalid escape sequence '\('
C:\Users\yhmg1v\AppData\Local\Temp\ipykernel_23200\3542194344.py:13: SyntaxWarning: invalid escape sequence '\('
  "content": """


###############################################################################################################

Certainly! Below is an organized extraction of **agent behaviors** along with their **corresponding mathematical formulas** as stated in the description. The behaviors are grouped by agent type: **Households**, **Realtors**, **Houses**, and **Records**.

---

## 1. Households

### Initialization Behaviors

- **Calculate initial capital:**

\[
c_{i,t_0} = y_{i,t_0} \times \eta_i
\]

where:
- \(c_{i,t_0}\): initial capital of household \(i\),
- \(y_{i,t_0}\): income of household \(i\) at initialization,
- \(\eta_i\): ratio factor relating capital to income.

- **Mortgage households select mortgage house and rent houses to own, update ownership state.**

- **Calculate highest affordable mortgage repayment:**

\[
a_{i,t_0} = \frac{y_{i,t_0} \times \alpha_i}{s}
\]

- **Calculate maximum mortgage they can pay (repayment formula):**

\[
M_{i,t_0} = \frac{a_{i,t_0}}{I_{t_0}} \times 

Reformat the conversation (older version of `track_completions` did not generate this as an output)

In [20]:
## reformat the conversation
conversation = list()
for i in range(len(completions["reply"])):
    conversation.append(completions["prompt"][i])
    conversation.append(
        {
            "role": "assistant",
            "content": completions["reply"][i]
        }
        )
print(conversation[1])

{'role': 'assistant', 'content': 'Thank you for sharing the comprehensive Model Description and equations again.\n\nYou previously requested a **Python implementation** that includes **all agent behaviors and formulas** with a **game-like UI using pygame**, and a graph with average sale and rent prices.\n\n---\n\n### Here is a full Python model implementation matching your description and equations precisely:\n\n- Each **time step = 3 months** (i.e., steps per year = 4).\n- All **state variables**, **agent behaviors**, and **decisions** directly derived from your equations.\n- Agents: **Households** (mortgage/rent types), **Realtors**, and **Houses** with full interactions.\n- Market clearing with house chains solved.\n- Shows a **pygame visualization of the grid** and **matplotlib graph for prices** on the right.\n- Careful implementation of financial formulas Eq.(1)–(17).\n\n---\n\n## Complete Python ABM with pygame UI code\n\n```python\nimport pygame\nimport numpy as np\nimport rand